# Construction of a deep neural network for predicting insurance claim probabilities

In this tutorial we will build an deep neural network (3 layers) to predict the probabilities of customers filing insurance claims. I am doing the preprocessing in pandas and scaling the data using scikit learn's StandardScaler function.

Imports for the functions we use:

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

Next we read in the data.

In [2]:

test_dat = pd.read_csv('test.csv')
train_dat = pd.read_csv('train.csv')
submission = pd.read_csv('sample_submission.csv')


## Cleaning the data

We split the y values off to a separate object, and drop the ids and targets from the train and test dataframes. Since we will be manipulating the data, we need to merge the train and test dataframes so that we are making the same changes to each dataset (otherwise predictions would be impossible).

In [3]:

train_y = train_dat['target']
train_x = train_dat.drop(['target', 'id'], axis = 1)
test_dat = test_dat.drop(['id'], axis = 1)

merged_dat = pd.concat([train_x, test_dat],axis=0)


Below we make several changes to the data prior to training the neural network. First the inputs are changed to float32 (as float64 is not compatabile with all of tensorflow's models). Second we one-hot encode the categorical variables. Thirdly, we standardize the scale of the numerical variables.

In [4]:
#change data to float32
for c, dtype in zip(merged_dat.columns, merged_dat.dtypes): 
    if dtype == np.float64:     
        merged_dat[c] = merged_dat[c].astype(np.float32)

#one hot encode the categoricals
cat_features = [col for col in merged_dat.columns if col.endswith('cat')]
for column in cat_features:
    temp=pd.get_dummies(pd.Series(merged_dat[column]))
    merged_dat=pd.concat([merged_dat,temp],axis=1)
    merged_dat=merged_dat.drop([column],axis=1)

#standardize the scale of the numericals
numeric_features = [col for col in merged_dat.columns if '_calc_' in  str(col)]
numeric_features = [col for col in numeric_features if '_bin' not in str(col)]

scaler = StandardScaler()
scaled_numerics = scaler.fit_transform(merged_dat[numeric_features])
scaled_num_df = pd.DataFrame(scaled_numerics, columns =numeric_features )


With the data munged, we can now split it back into train and test variables

In [5]:

merged_dat = merged_dat.drop(numeric_features, axis=1)

merged_dat = np.concatenate((merged_dat.values,scaled_num_df), axis = 1)

train_x = merged_dat[:train_x.shape[0]]
test_dat = merged_dat[train_x.shape[0]:]



## Training the neural network

Below we ste up the neural network in tensorflow, and fit the training data.

In [6]:
config = tf.contrib.learn.RunConfig(tf_random_seed=42) # not shown in the config

#create a set of real valued columns from training set
feature_cols = tf.contrib.learn.infer_real_valued_columns_from_input(train_x)

We then create the DNN classifier. DNN == deep neural network

hidden units means the number of units per layer, of the neural network abd all layers fully connected.
'[64,32]' would be 64 nodes in first layer and 32 in second.

In [7]:
dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[150,150,150], n_classes=2,
                                         feature_columns=feature_cols, config=config)


INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x120d36470>, '_master': '', '_num_ps_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': 42, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000}


Fitting the model is just like with SciKit learn, the steps variable indicates the number of training iterations and the batch_size is the number of samples used to train the network on each step.

In [8]:
dnn_clf.fit(train_x, train_y, batch_size=50, steps=40000)


Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scal

INFO:tensorflow:loss = 0.146951, step = 6801
INFO:tensorflow:global_step/sec: 178.923
INFO:tensorflow:loss = 0.161164, step = 6901
INFO:tensorflow:global_step/sec: 195.943
INFO:tensorflow:loss = 0.280902, step = 7001
INFO:tensorflow:global_step/sec: 187.397
INFO:tensorflow:loss = 0.328847, step = 7101
INFO:tensorflow:global_step/sec: 167.458
INFO:tensorflow:loss = 0.218686, step = 7201
INFO:tensorflow:global_step/sec: 188.284
INFO:tensorflow:loss = 0.0451066, step = 7301
INFO:tensorflow:global_step/sec: 180.998
INFO:tensorflow:loss = 0.112401, step = 7401
INFO:tensorflow:global_step/sec: 154.365
INFO:tensorflow:loss = 0.039594, step = 7501
INFO:tensorflow:global_step/sec: 184.834
INFO:tensorflow:loss = 0.159198, step = 7601
INFO:tensorflow:global_step/sec: 190.524
INFO:tensorflow:loss = 0.157674, step = 7701
INFO:tensorflow:global_step/sec: 188.016
INFO:tensorflow:loss = 0.116975, step = 7801
INFO:tensorflow:global_step/sec: 177.368
INFO:tensorflow:loss = 0.242035, step = 7901
INFO:ten

INFO:tensorflow:global_step/sec: 167.467
INFO:tensorflow:loss = 0.281723, step = 16301
INFO:tensorflow:global_step/sec: 167.152
INFO:tensorflow:loss = 0.36385, step = 16401
INFO:tensorflow:global_step/sec: 166.194
INFO:tensorflow:loss = 0.438149, step = 16501
INFO:tensorflow:global_step/sec: 186.904
INFO:tensorflow:loss = 0.107105, step = 16601
INFO:tensorflow:global_step/sec: 184.921
INFO:tensorflow:loss = 0.234758, step = 16701
INFO:tensorflow:global_step/sec: 190.435
INFO:tensorflow:loss = 0.0958604, step = 16801
INFO:tensorflow:global_step/sec: 178.172
INFO:tensorflow:loss = 0.152733, step = 16901
INFO:tensorflow:global_step/sec: 165.453
INFO:tensorflow:loss = 0.331189, step = 17001
INFO:tensorflow:global_step/sec: 114.398
INFO:tensorflow:loss = 0.222139, step = 17101
INFO:tensorflow:global_step/sec: 157.506
INFO:tensorflow:loss = 0.171572, step = 17201
INFO:tensorflow:global_step/sec: 183.068
INFO:tensorflow:loss = 0.160443, step = 17301
INFO:tensorflow:global_step/sec: 139.287
IN

INFO:tensorflow:loss = 0.0340932, step = 25701
INFO:tensorflow:global_step/sec: 186.767
INFO:tensorflow:loss = 0.163491, step = 25801
INFO:tensorflow:global_step/sec: 190.531
INFO:tensorflow:loss = 0.0389575, step = 25901
INFO:tensorflow:global_step/sec: 188.591
INFO:tensorflow:loss = 0.345814, step = 26001
INFO:tensorflow:global_step/sec: 181.322
INFO:tensorflow:loss = 0.290721, step = 26101
INFO:tensorflow:global_step/sec: 174.37
INFO:tensorflow:loss = 0.153903, step = 26201
INFO:tensorflow:global_step/sec: 194.188
INFO:tensorflow:loss = 0.172393, step = 26301
INFO:tensorflow:global_step/sec: 177.45
INFO:tensorflow:loss = 0.0943331, step = 26401
INFO:tensorflow:global_step/sec: 184.986
INFO:tensorflow:loss = 0.147896, step = 26501
INFO:tensorflow:global_step/sec: 177.276
INFO:tensorflow:loss = 0.0301194, step = 26601
INFO:tensorflow:global_step/sec: 180.053
INFO:tensorflow:loss = 0.0395721, step = 26701
INFO:tensorflow:global_step/sec: 184.865
INFO:tensorflow:loss = 0.1117, step = 26

INFO:tensorflow:loss = 0.106048, step = 35101
INFO:tensorflow:global_step/sec: 168.344
INFO:tensorflow:loss = 0.117411, step = 35201
INFO:tensorflow:global_step/sec: 169.638
INFO:tensorflow:loss = 0.113005, step = 35301
INFO:tensorflow:global_step/sec: 190.312
INFO:tensorflow:loss = 0.101606, step = 35401
INFO:tensorflow:global_step/sec: 142.356
INFO:tensorflow:loss = 0.161154, step = 35501
INFO:tensorflow:global_step/sec: 156.04
INFO:tensorflow:loss = 0.17195, step = 35601
INFO:tensorflow:global_step/sec: 179.728
INFO:tensorflow:loss = 0.210832, step = 35701
INFO:tensorflow:global_step/sec: 155.611
INFO:tensorflow:loss = 0.172585, step = 35801
INFO:tensorflow:global_step/sec: 178.298
INFO:tensorflow:loss = 0.217573, step = 35901
INFO:tensorflow:global_step/sec: 187.003
INFO:tensorflow:loss = 0.165332, step = 36001
INFO:tensorflow:global_step/sec: 180.833
INFO:tensorflow:loss = 0.099917, step = 36101
INFO:tensorflow:global_step/sec: 161.65
INFO:tensorflow:loss = 0.263736, step = 36201


DNNClassifier(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._BinaryLogisticHead object at 0x1249ae8d0>, 'hidden_units': [150, 150, 150], 'feature_columns': (_RealValuedColumn(column_name='', dimension=227, default_value=None, dtype=tf.float64, normalizer=None),), 'optimizer': None, 'activation_fn': <function relu at 0x113e61598>, 'dropout': None, 'gradient_clip_norm': None, 'embedding_lr_multipliers': None, 'input_layer_min_slice_size': None})

## Predicting probabilities with the neural network

We then predict the probabilities using predict proba. A generator object is produced, so I wrap it in a list function so that we can move to list form (and easily put the data into a pandas df). 

In [9]:
dnn_y_pred = dnn_clf.predict_proba(test_dat)

dnn_out = list(dnn_y_pred)


Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


Note the predicted probabilities are provided in pairs of (P[0], P[1]). We want the probability of a claim (P[1]) so I use a list comprehension below to grab only the second member of the array when I add the data to the output dataframe.

In [10]:
dnn_output = submission
dnn_output['target'] = [x[1] for x in dnn_out]

dnn_output.to_csv('dnn_predictions2.csv', index=False, float_format='%.4f')